In [ ]:
import xlwings as xw
from xlwings import view
import pandas as pd

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import pandas as pd
import os,read

In [ ]:
import importlib,utils,json,os
from utils import *

In [ ]:

import importlib,utils,json,os
from utils import *
gpt_eval=utils.read_json("gpt_eval_3000.json")



In [ ]:
os.environ['OPENAI_API_KEY'] = read.read_json(os.path.join('configuration.json'))['OpenAI_api_key']

In [ ]:

def get_formular(question: str,table, model='gpt-3.5-turbo'):
    llm = ChatOpenAI(model_name=model, max_tokens=50,)
    prompt = "Write a excel formula to answer the quesion based on table beleow\n"\
        "You can use functions like SUM,XLOOKUP,UNIQUE,AVERAGE,etc...\n"\
             "{question}\n" \
             "Here is the table: {table}.\n" \
             "{format_instructions}"
             
    prompt_template = ChatPromptTemplate.from_template(prompt)
    options = ResponseSchema(name="Formula", description="The formula is related to question")
    response_schemas = [options]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    form_instructions = output_parser.get_format_instructions()
    
    final_prompt = prompt_template.format_messages(question=question, table=table, format_instructions=form_instructions)
    print(final_prompt)
    output = llm(final_prompt)
    return output_parser.parse(output.content)



In [ ]:
gpt_eval=utils.read_json("gpt_eval_direct.json")

In [ ]:
gpt_eval[1].keys()

In [ ]:
import csv,io

def list_to_csv(data, file_path):
    with open(file_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerows(data)


file_path_f = 'output{i}.csv'
# Pass a query to the data
for i,item in enumerate(gpt_eval):
    if col_name not in item.keys():
        print(i)
        file_path = file_path_f.format(i=i)
        data=item["Table"]

        list_to_csv(data, file_path)

        loader = CSVLoader(file_path)  # Pass the in-memory file-like object


        # Create an index using the loaded documents
        index_creator = VectorstoreIndexCreator()
        docsearch = index_creator.from_loaders([loader])
        # Create a question-answering chain using the index
        chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")
        query=item ["Question"]
        print(query)
        response = chain({"question": query})
        item[col_name]=response['result']
        ans=item ["gdres"]
        print(response['result'],ans)

In [ ]:
import concurrent.futures

def process_item(i, item):
    try:
        if col_name not in item.keys():
            file_path = file_path_f.format(i=i)
            data = item["Table"]

            list_to_csv(data, file_path)

            loader = CSVLoader(file_path) 

            index_creator = VectorstoreIndexCreator()
            docsearch = index_creator.from_loaders([loader])
            chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")
            query = item["Question"]
            response = chain({"question": query})
            item[col_name]=response['result']
            ans = item["gdres"]
            print(response['result'],ans)
        return item
    except Exception as e:
        print(f"Error processing item {i}: {e}")
        return None


# Define the number of worker processes to use
num_workers = 4

# Create a ProcessPoolExecutor
with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
    gpt_eval = list(executor.map(process_item, range(len(gpt_eval)), gpt_eval))


In [ ]:
with open('gpt_eval_direct.json', 'r') as f:
    res_list=json.load(f)

In [ ]:
with open('gpt_eval_agents.json', 'r') as f:
    res_list=json.load(f)

In [ ]:
from thefuzz import fuzz
from thefuzz import process

def fuzzy_cmp_str(a:str,b:str):
    a,b=str(a).lower(),str(b).lower()
    return fuzz.partial_ratio(a, b)

In [ ]:
thres=95
count=0
col_name="LC_agent_res"
col_name="LC_res"

for i,item in enumerate(res_list):
    
    item['res']="failed"
    radio=fuzzy_cmp_str(res_list[i][col_name],res_list[i]["gdres"])
    #print(radio)
    if(radio>thres):
        count+=1
        item['res']="exe_match"
        print(res_list[i][col_name],"||",res_list[i]["gdres"])
    else:
        item['res']="failed"
print(count/len(res_list))

In [ ]:
thres=95
count=0
col_name="LC_agent_res"
col_name="llm"

for i,item in enumerate(res_list):
    
    item['res']="failed"
    radio=fuzzy_cmp_str(res_list[i][col_name],res_list[i]["gdres"])
    #print(radio)
    if(radio>thres):
        count+=1
        item['res']="exe_match"
        print(res_list[i][col_name],"||",res_list[i]["gdres"])
    else:
        item['res']="failed"
print(count/len(res_list))

In [ ]:

# 初始化计数器
counters = {
    'easy': {'success': 0, 'failed': 0},
    'medium': {'success': 0, 'failed': 0},
    'hard': {'success': 0, 'failed': 0},
    'Calculation': {'success': 0, 'failed': 0},
}

# 统计计数
for item in res_list:
    level = item['Level']
    result = item['res']
    if result == 'exact_match' or result == 'exe_match':
        counters[level]['success'] += 1
    elif result == 'failed':
        counters[level]['failed'] += 1

# 打印结果
for level, results in counters.items():
    print(f"Level: {level}")
    total = sum(results.values())
    for result, count in results.items():
        percentage = (count / total) * 100
        print(f"{result}: {count} ({percentage:.2f}%)")


In [ ]:
df_res=pd.DataFrame(counters).T
df_res["rate"]=df_res['success']/(df_res['success']+df_res["failed"])
df_res

In [ ]:
df_res['success'].sum()/(df_res['success'].sum()+df_res['failed'].sum())